# Imports

In [2]:
import re
import pandas as pd
from collections import defaultdict
import csv
from sklearn.linear_model import LinearRegression
# %load_ext tensorboard
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.layers as Layer
import tensorboard
import matplotlib.pyplot as plt
import numpy as np
# import keras_tuner as kt
import time
import tensorflow.keras.layers.experimental.preprocessing as preprocessing

# Data Formatting & Manipulation

In [3]:

movieFields = ['id', 'title', 'vote_average', 'vote_count', 'genres']
castFields = ['id', 'cast']

movieData = pd.read_csv('archive/movies_metadata.csv', skipinitialspace=True, usecols=movieFields)
castData = pd.read_csv('archive/credits.csv', skipinitialspace=True, usecols=castFields)

dfMovies = pd.DataFrame(movieData)
dfCast = pd.DataFrame(castData)

df = pd.merge(dfMovies, dfCast, how='inner')


# ---------------------------------------------------------------------------- #
# The following section formats the cast and creates a clean list of actor names.

unformattedNames = df['cast']

# main list of cast grouped by movies
cast_master_copy = []

# list of overall cast by individual names
actor_name_list = []

# reads through the cast string for each movie
for unformattedNameString in unformattedNames:

    # creates new list for current movie
    grouped = []

    # pulls actors' names out of the paragraph
    line_list = re.findall("(?<=\'name\': )(.*?)(?=,)", unformattedNameString)
    
    # converts list of cast for one movie into a string to manipulate further 
    line_string = str(line_list)

    # separates cast into a unique list per movie
    line_split = line_string.split(", ")

    actor_count = 0
    # reads through each actor per movie
    for name in line_split:
        if (actor_count == 5):
            break
     
        # removes extraneous symbols from the actors' names
        name = name.replace("\"", "")
        name = name.replace("\'", "")
        name = name.replace("[", "")
        name = name.replace("]", "")

        # adds actors to two working lists
        # grouped: actors are in lists by movies
        # actor_name_list: puts every actors in one list
        grouped.append(name)
        actor_name_list.append(name)
        
        actor_count += 1

    # adds formatted cast members to list
    cast_master_copy.append(grouped)

# assigns formatted cast to 'cast' column of DataFrame
df['cast'] = cast_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section turns the list of actor names into a list of unique numbers
# assigned to those actors. Then, creates a dictionary with the actor names and those 
# keys.

# initializes default dictionary
key_assignment = defaultdict(lambda: len(key_assignment))

# list of unique keys for each actor in dictionary
keys = [key_assignment[key] for key in actor_name_list]

# assigns keys to actors to form a hash table...nums are keys
actor_dict = dict(zip(keys, actor_name_list))

# inverts dictionary...names are keys
actor_dict_inv = {v: k for k, v in actor_dict.items()}
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section uses the inverted actor dictionary to look up the actor names
# and put their respective dictionary values in a list for each movie.

cast_names_by_movie = df['cast']
cast_ids_by_movie_master = []

for cast in cast_names_by_movie:

    temp_actor_list = []

    for actor in cast:
        actor = actor.replace("[", "")
        actor = actor.replace("]", "")
        actor = actor.replace("\'", "")

        actor_key = actor_dict_inv.get(actor)
        temp_actor_list.append(actor_key)
    
    if (len(temp_actor_list) < 5):
            diff = 5 - len(temp_actor_list)
            
            for i in range(diff):
                temp_actor_list.append(1)

    cast_ids_by_movie_master.append(temp_actor_list)

df['cast_ids'] = cast_ids_by_movie_master
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section formats the genre and adds it to the data frame

unformatted_genres = df['genres']

# main list of genres grouped by movies
genre_master_copy = []

# list of overall cast by individual names
genre_name_list = []

# reads through the genre string for each movie
for unformatted_genre_string in unformatted_genres:

    # creates new list for current movie
    genre_grouped = []

    # pulls genres out of the paragraph
    genre_line_list = re.findall("(?<=\'name\': )(.*?)(?=})", unformatted_genre_string)

    # converts list of genres for one movie into a string to manipulate further 
    genre_line_string = str(genre_line_list)

    # separates genre into a unique list per movie
    genre_line_split = genre_line_string.split(", ")

    # reads through each genre per movie & grabs the first one
    for genre in genre_line_split:
     
        # removes extraneous symbols from the genres
        genre = genre.replace("\"", "")
        genre = genre.replace("\'", "")
        genre = genre.replace("[", "")
        genre = genre.replace("]", "")

        genre_str = str(genre)
        break

    # adds formatted genre to list
    genre_master_copy.append(genre_str)

# assigns formatted cast to 'cast' column of DataFrame
df['genres'] = genre_master_copy
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section filters out data points that will be harmful for
# our model's accuracy.

# gets rid of all movies with 0 votes
# Number of movies before operation: 43020
# Number of movies after operation: 40739
df = df.drop(df[df.vote_count < 10].index)
# ---------------------------------------------------------------------------- #


# ---------------------------------------------------------------------------- #
# The following section handles the creation of the .csv files that will be the
# primary data used to train the models. 

# writes formatted DataFrame values to a new csv
# pd.DataFrame.to_csv(df, "formatted_movies.csv", index="false")

# creates a csv of the actor dictionary
# w = csv.writer(open("actor_dict.csv", "w"))
# for key, val in actor_dict.items():
#     w.writerow([key, val])
# ---------------------------------------------------------------------------- #

# print(df)


# Preparing Training Dataset

In [4]:
np.random.seed(1)
tf.random.set_seed(1)

movie_df = df
movie_df.drop(['vote_count'], axis = 1)

# saves genres keys/values if needed for lookup later
keys, genres = pd.factorize(movie_df['genres'])

# print(genres.shape)
# movie_df['genres'] = pd.factorize(movie_df['genres'])[0]

data_count = movie_df.shape[0]
training_data_count = int(data_count * 0.75) + 1
test_data_count = data_count - training_data_count

y_train = np.array(movie_df['vote_average'].head(training_data_count))
x_train = np.array(movie_df[['genres']].head(training_data_count))

y_test = np.array(movie_df['vote_average'].tail(test_data_count))
x_test = np.array(movie_df[['genres']].tail(test_data_count))

print(x_train)
print(x_train.shape)

# print(x_test)
# print(x_test.shape)

[['Horror']
 ['Action']
 ['Action']
 ...
 ['Crime']
 ['Drama']
 ['Adventure']]
(16989, 1)


# Text Vectorization (Genres)

In [5]:
VOCAB_SIZE = 20
x_test_list = x_test.tolist()
encoder = preprocessing.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(x_test_list)

vocab = np.array(encoder.get_vocabulary())
vocab

Metal device set to: Apple M1 Max

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB



2021-11-05 18:51:09.177364: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-05 18:51:09.177497: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2021-11-05 18:51:09.216425: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-05 18:51:09.216603: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-05 18:51:09.249825: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


array(['', '[UNK]', 'drama', 'comedy', 'action', 'adventure', 'horror',
       'crime', 'thriller', 'fantasy', 'animation', 'romance', 'mystery',
       'documentary', 'science', 'fiction', 'family', 'music', 'western',
       'war'], dtype='<U11')

## Text Vectorization Sanity Check (Genres)

In [6]:
encoded_example = encoder(movie_df.genres.to_numpy()[0]).numpy()
print(movie_df.genres.to_numpy()[0])
encoded_example

Horror


array([6])

# Recurrent Neural Network

In [7]:
model_RNN = tf.keras.Sequential([
 encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=100,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.SimpleRNN(100),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1)
])

model_RNN.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(1e-3),
              metrics=['accuracy'])

model_RNN.summary()

# # Define the Keras TensorBoard callback.
# logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
# tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

# # Train the model.
model_RNN.fit(
    x_train,
    y_train, 
    batch_size=512,
    epochs=25)

# # Evaluate
score = model_RNN.evaluate(x_train, y_train)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, None)              0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         2000      
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 100)               20100     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 32,301
Trainable params: 32,301
Non-trainable params: 0
_________________________________________________________________
Epoch 1/25


2021-11-05 18:51:09.769927: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


34/34 [==============================] - 1s 25ms/step - loss: 25.0211 - accuracy: 5.8862e-05
Epoch 2/25
34/34 [==============================] - 1s 24ms/step - loss: 1.9164 - accuracy: 0.0000e+00
Epoch 3/25
34/34 [==============================] - 1s 23ms/step - loss: 1.1538 - accuracy: 0.0000e+00
Epoch 4/25
34/34 [==============================] - 1s 23ms/step - loss: 1.1262 - accuracy: 0.0000e+00
Epoch 5/25
34/34 [==============================] - 1s 24ms/step - loss: 1.1260 - accuracy: 0.0000e+00
Epoch 6/25
34/34 [==============================] - 1s 23ms/step - loss: 1.1260 - accuracy: 0.0000e+00
Epoch 7/25
34/34 [==============================] - 1s 24ms/step - loss: 1.1249 - accuracy: 0.0000e+00
Epoch 8/25
34/34 [==============================] - 1s 23ms/step - loss: 1.1260 - accuracy: 0.0000e+00
Epoch 9/25
34/34 [==============================] - 1s 24ms/step - loss: 1.1223 - accuracy: 0.0000e+00
Epoch 10/25
34/34 [==============================] - 1s 25ms/step - loss: 1.1229 - 

2021-11-05 18:51:30.276751: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


531/531 [==============================] - 4s 7ms/step - loss: 1.0807 - accuracy: 0.0000e+00
Test loss: 1.0807479619979858
Test accuracy: 0.0


In [11]:
sample_text = ('action')
predictions = model_RNN.predict(np.array([sample_text]))
print(predictions[0])

[5.811191]
